In [17]:
latest_gameweek = 34

In [18]:
import pandas as pd
import numpy as np
from pathlib import Path

import gradio as gr
from scipy.ndimage import gaussian_filter

import plotly.graph_objects as go
import seaborn as sns
sns.set_style("darkgrid")


# Data

In [19]:
# POSITION MAPPING
position_dict = {1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}

In [20]:
filepath = Path(f'../data/predictions/gameweek{latest_gameweek}.csv')
projections = pd.read_csv(filepath, index_col=0)
projections['position'] = projections['element_type'].map(position_dict)
display(projections.head())
display(projections.shape)

,name,element_type,home,corners_and_indirect_freekicks_order,creativity_rank,direct_freekicks_order,ict_index_rank,influence_rank,minutes,now_cost,...,gameweek_xA_expanding_per90,gameweek_xGA_expanding_per90,gameweek_xPoints_expanding_per90,xG_overperformance,team_name,opponent_team,date,gameweek,expected_points,position
0,Aaron Cresswell,2,1,3.0,338,NaN,446,449,321,42,...,0.141730,1.331404,3.627780,1.950078,West Ham,Liverpool,2024-04-27T11:30:00Z,35,1.588187,DEF
1,Aaron Cresswell,2,0,3.0,338,NaN,446,449,321,42,...,0.141730,1.331404,3.627780,1.950078,West Ham,Chelsea,2024-05-05T13:00:00Z,36,1.668047,DEF
2,Aaron Cresswell,2,1,3.0,338,NaN,446,449,321,42,...,0.141730,1.331404,3.627780,1.950078,West Ham,Luton,2024-05-11T14:00:00Z,37,2.597269,DEF
3,Aaron Cresswell,2,0,3.0,338,NaN,446,449,321,42,...,0.141730,1.331404,3.627780,1.950078,West Ham,Manchester City,2024-05-19T15:00:00Z,38,1.419396,DEF
4,Aaron Hickey,2,0,NaN,272,NaN,283,256,713,45,...,0.020959,1.229178,3.259115,0.000000,Brentford,Everton,2024-04-27T16:30:00Z,35,2.229651,DEF


(2460, 114)

In [21]:
filepath = Path('../data/fpl_df.csv')
fpl_df = pd.read_csv(filepath, index_col=0, low_memory=False)
fpl_df = fpl_df[fpl_df.season=='23-24']
display(fpl_df.head())
display(fpl_df.shape)

,assists,bonus,bps,clean_sheets,corners_and_indirect_freekicks_order,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,dreamteam_count,...,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,name,data_retrieved_datetime
18839,0,0,2,0,NaN,0.0,554,210,NaN,0,...,218.0,77.0,218.0,77.0,7.0,3.0,0.00,0.00,Gabriel dos Santos Magalhães,2023-08-14 21:41:02.445217
18840,0,0,11,0,NaN,30.3,22,16,NaN,0,...,156.0,73.0,156.0,73.0,76.0,25.0,1.00,0.00,Kai Havertz,2023-08-14 21:41:02.445217
18841,1,0,23,0,4.0,42.5,11,7,3.0,0,...,51.0,16.0,51.0,16.0,29.0,8.0,1.06,0.00,Gabriel Martinelli Silva,2023-08-14 21:41:02.445217
18842,0,2,26,1,NaN,5.9,110,10,NaN,0,...,17.0,5.0,17.0,5.0,88.0,16.0,1.25,1.25,Eddie Nketiah,2023-08-14 21:41:02.445217
18843,0,0,11,0,3.0,23.4,39,27,1.0,0,...,158.0,74.0,158.0,74.0,14.0,6.0,1.00,0.00,Martin Ødegaard,2023-08-14 21:41:02.445217


(9780, 209)

In [22]:
df = fpl_df.groupby('name').last().reset_index()[['name', 'team_name', 'element_type', 'now_cost', 
                            'gameweek_minutes_ewm_20', 'points_per_game', 'total_points', 
                            'gameweek_xPoints_ewm_5', 'gameweek_xPoints_ewm_10', 'gameweek_xPoints_ewm_20', 'gameweek_xPoints_ewm_40']]
df['games_played'] = np.round(np.where(df['points_per_game']!=0, df['total_points'] / df['points_per_game'], 0),0)
df['price'] = df['now_cost'] / 10.0
df['value'] = df['gameweek_xPoints_ewm_20'] / df['price']
df['value_points'] = np.sqrt( df['gameweek_xPoints_ewm_20'] *  df['value'])

# EXPECTED POINTS
expected_points_next_10gw = (projections[projections.gameweek.isin( np.arange(latest_gameweek+1, latest_gameweek+11, 1) )]
 .groupby('name')
 .sum()
 )[['expected_points']].reset_index().rename(columns={'expected_points':'expected_points_next_10_GW'})

expected_points_next_5gw = (projections[projections.gameweek.isin( np.arange(latest_gameweek+1, latest_gameweek+6, 1) )]
 .groupby('name')
 .sum()
 )[['expected_points']].reset_index().rename(columns={'expected_points':'expected_points_next_5_GW'})

df = df.merge(expected_points_next_10gw, on='name', how='left')
df = df.merge(expected_points_next_5gw, on='name', how='left')

# POSITION MAPPING
df['position'] = df['element_type'].map(position_dict)

display(df)

,name,team_name,element_type,now_cost,gameweek_minutes_ewm_20,points_per_game,total_points,gameweek_xPoints_ewm_5,gameweek_xPoints_ewm_10,gameweek_xPoints_ewm_20,gameweek_xPoints_ewm_40,games_played,price,value,value_points,expected_points_next_10_GW,expected_points_next_5_GW,position
0,Aaron Cresswell,West Ham,2,42,65.710651,0.9,7,1.769273,2.369443,2.772282,2.987950,8.0,4.2,0.660067,1.352735,7.272900,7.272900,DEF
1,Aaron Hickey,Brentford,2,45,77.587197,1.9,17,2.526123,2.707357,2.819039,2.877821,9.0,4.5,0.626453,1.328908,9.102537,9.102537,DEF
2,Aaron Ramsdale,Arsenal,1,45,90.000000,3.3,20,3.789068,4.093285,4.399517,4.476464,6.0,4.5,0.977670,2.073952,15.206312,15.206312,GK
3,Aaron Ramsey,Burnley,3,50,38.760392,1.3,18,1.644932,1.583594,1.570526,1.569442,14.0,5.0,0.314105,0.702361,7.272721,7.272721,MID
4,Aaron Wan-Bissaka,Manchester Utd,2,44,82.225925,2.2,35,2.053557,2.478524,2.934714,3.247205,16.0,4.4,0.666980,1.399070,12.048719,12.048719,DEF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,Youssef Ramalho Chermiti,Everton,4,48,4.922454,0.8,10,0.734728,0.874055,0.923552,0.942296,12.0,4.8,0.192407,0.421542,5.395854,5.395854,FWD
554,Yves Bissouma,Tottenham,3,50,74.903218,1.6,38,1.691171,1.874199,2.052937,2.185656,24.0,5.0,0.410587,0.918101,11.863018,11.863018,MID
555,Zack Nelson,Luton,3,45,17.785428,1.0,2,1.619647,1.488610,1.477886,1.495103,2.0,4.5,0.328419,0.696682,5.834730,5.834730,MID
556,Zeki Amdouni,Burnley,4,52,54.521238,2.5,74,1.386755,1.872864,2.161365,2.291115,30.0,5.2,0.415647,0.947821,8.328628,8.328628,FWD


# Gradio functions

In [23]:
def plot_points_and_value(positions, teams, show_names, max_price, x_axis_feature, y_axis_feature):
    
    fig = go.Figure()

    # aux df for manipulation
    df_out = df[df.position.isin(positions)].copy()
    # choose only given teams
    if "Select All" not in teams:
        df_out = df_out[df_out.team_name.isin(teams)]

    # drop players above max_price
    df_out = df_out[df_out.price<=max_price]

    # CREATE FIGURE
    fig.add_trace(
            go.Scatter(
                x=df_out[x_axis_feature],
                y=df_out[y_axis_feature],
                mode="markers+text",
                hovertext=df_out['name'].values,
                showlegend=False,
                ),
        )

    # add player names as visible
    if show_names:
        fig.update_traces(
            text = df_out['name'].values,
            textposition='top center',
            )

    # styling
    fig.update_layout(
        #title="",
        template='plotly_dark',
        xaxis_title=x_axis_feature,
        yaxis_title=y_axis_feature,
        #showlegend=True
    )

    return fig

In [24]:
def top_weekly_projections(positions, teams, gameweek):
    
    fig = go.Figure()

    # aux df for manipulation
    projections_out = projections[projections.position.isin(positions)].copy()
    # choose only given teams
    if "Select All" not in teams:
        projections_out = projections_out[projections_out.team_name.isin(teams)]
    # choose gameweek
    projections_out = projections_out[projections_out.gameweek==int(gameweek)]

    top_20 = projections_out.groupby('name').sum().sort_values(by='expected_points').reset_index().tail(20)

    # CREATE FIGURE
    fig.add_trace(
        go.Bar(
            x=top_20.expected_points,
            #y=np.arange(top_20.shape[0],0,-1), 
            y=top_20.name, 
            text=np.round(top_20.expected_points,2),
            textposition='outside',
            width=0.75,
            orientation='h'
        ),
    ) 

    # styling
    fig.update_layout(
        #title="",
        template='plotly_dark',
        xaxis_title='gameweek expected points',
        #yaxis_title='value',
        #showlegend=True
    )

    return fig

In [25]:
def player_points(players):

    marker_colors = ['red', 'blue']
    colors = ['rgba(255, 0, 0, 0.5)','rgba(0, 0, 255, 0.5)']

    fig = go.Figure()
    for count, player in enumerate(players):

        my_past_data = fpl_df[fpl_df['name']==player].sort_values(by='gameweek')
        my_projections = projections[projections['name']==player].sort_values(by='gameweek')
        
        x_past = list(np.unique(my_past_data['gameweek']))
        x_future = list(np.unique(my_projections['gameweek']))
        my_x = x_past + x_future

        y_past = list(my_past_data.groupby('gameweek').sum()['gameweek_xPoints'])
        y_future = list(my_projections.groupby('gameweek').sum()['expected_points'])
        my_y = y_past + y_future
        my_y_filtered = gaussian_filter(y_past + y_future, sigma=2, mode='nearest')

        fig.add_trace(
            go.Scatter(
                x=my_x,
                y=my_y_filtered,
                mode="markers+lines",
                marker=dict(color=marker_colors[count]),  
                line=dict(color=marker_colors[count], width=3),  
                fill='tozeroy',    
                fillcolor=colors[count],   
                name=player,            
                showlegend=True,
                ),
        )

        if len(players)==1:
            fig.add_trace(
                go.Scatter(
                    x=my_x,
                    y=my_y,
                    mode="markers",
                    marker=dict(color='white'),
                    name=player,            
                    showlegend=False,
                    ),
            )

    fig.add_vline(x=latest_gameweek+0.5,)

    fig.update_layout(
        #title="",
        template='plotly_dark',
        xaxis_title="gameweek",
        yaxis_title='expected points',
        #showlegend=True
    )

    return fig

In [26]:
#def player_points_swarm

# Demo

In [27]:
position_list = ['GK', 'DEF', 'MID', 'FWD']
team_name_list = ["Select All"]
team_name_list += list(np.sort(df.team_name.unique()))
gameweek_list = [str(x) for x in np.arange(latest_gameweek+1, latest_gameweek+11,)]
features = ["gameweek_xPoints_ewm_20", 'expected_points_next_10_GW', 'value']
x_axis_feature = features
y_axis_feature = features
minimum_price = df['price'].min()
maximum_price = df['price'].max()

In [28]:
scatter_demo = gr.Interface(
    plot_points_and_value,
    [
        gr.CheckboxGroup(position_list, label="POSITION", value=position_list),
        gr.Dropdown(team_name_list, label="TEAM", multiselect=True, value="Select All"),
        gr.Checkbox('Show player names', value=False),
        gr.Slider(minimum_price, maximum_price, value=maximum_price, info='Choose maximum allowed player value.'),
        gr.Dropdown(x_axis_feature, label="x-axis", value='gameweek_xPoints_ewm_20'),
        gr.Dropdown(y_axis_feature, label="y-axis", value = 'value'),
    ],
    gr.Plot(),
)

In [29]:
weekly_top_players_demo = gr.Interface(
    top_weekly_projections,
    [
        gr.CheckboxGroup(position_list, label="POSITION", value=position_list),
        gr.Dropdown(team_name_list, label="TEAM", multiselect=True, value="Select All"),
        gr.Dropdown(gameweek_list, label="GAMEWEEK", value=str(latest_gameweek+1)),
    ],
    gr.Plot(),
)

In [30]:
player_name_list = list(fpl_df.name.unique())

In [31]:
player_points_demo = gr.Interface(
    player_points,
    [
        gr.Dropdown(player_name_list, value='Erling Haaland', multiselect=True, max_choices=2, label='Choose 1 or 2 players')
    ],
    gr.Plot(),
)

In [32]:
full_demo = gr.TabbedInterface(
    [scatter_demo, weekly_top_players_demo, player_points_demo],
    ['Scatter plots', 'Gameweek projections top 20', 'Player xPoints and projections']
).launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


# DEV STUFF

In [ ]:
teams = ['Aston Villa']
positions = ['MID']

In [ ]:
fig = go.Figure()

df_out = df[df.position.isin(positions)].copy()
if "Select All" not in teams:
    df_out = df_out[df_out.team_name.isin(teams)]

fig.add_trace(
    go.Scatter(
        x=df_out['gameweek_xPoints_ewm_20'],
        y=df_out['value'],
        mode="markers+text",
        text = df_out['name'].values,
        hovertext=df_out['name'].values,
        showlegend=False,
        ),
)

fig.update_traces(textposition='top center')

fig.update_layout(
    #title="",
    template='plotly_dark',
    xaxis_title="gameweek_xPoints_ewm_20",
    yaxis_title='value',
    #showlegend=True
)

